In [1]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!wget https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv -O titanic.csv

--2025-06-10 17:31:11--  https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv
Resolving web.stanford.edu (web.stanford.edu)... 171.67.215.200, 2607:f6d0:0:925a::ab43:d7c8
Connecting to web.stanford.edu (web.stanford.edu)|171.67.215.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44225 (43K) [text/csv]
Saving to: ‘titanic.csv’

100%[======================================>] 44,225      --.-K/s   in 0.1s    

2025-06-10 17:31:12 (416 KB/s) - ‘titanic.csv’ saved [44225/44225]



In [3]:
import pandas as pd

df = pd.read_csv("titanic.csv")
print(df.shape)
print(df.columns.tolist())

(887, 8)
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///titanic.db")
df.to_sql("titanic", engine, index=False)

887

In [5]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM titanic WHERE Age < 2;"))

sqlite
['titanic']
[(1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 1, 'Master. Hudson Trevor Allison', 'male', 0.92, 1, 2, 151.55), (1, 3, 'Miss. Maria Nakid', 'female', 1.0, 0, 2, 15.7417), (0, 3, 'Master. Sidney Leonard Goodwin', 'male', 1.0, 5, 2, 46.9), (1, 3, 'Miss. Helene Barbara Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 3, 'Miss. Eugenie Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 2, 'Master. Viljo Hamalainen', 'male', 0.67, 1, 1, 14.5), (1, 3, 'Master. Bertram Vere Dean', 'male', 1.0, 1, 2, 20.575), (1, 3, 'Master. Assad Alexander Thomas', 'male', 0.42, 0, 1, 8.5167), (1, 2, 'Master. Andre Mallet', 'male', 1.0, 0, 2, 37.0042), (1, 2, 'Master. George Sibley Richards', 'male', 0.83, 1, 1, 18.75)]


In [9]:
# Ensure your AWS credentials are configured

from langchain.chat_models import init_chat_model

llm = init_chat_model("us.anthropic.claude-3-5-sonnet-20240620-v1:0", model_provider="bedrock_converse")

In [10]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [11]:
agent_executor.invoke({"input": "what's the average age of survivors"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded: [{'type': 'text', 'text': "  Finally, I can write and execute a SQL query to answer the question about the average age of survivors.\n\nFirst, let's list the tables in the database:", 'index': 0}, {'type': 'tool_use', 'name': 'sql_db_list_tables', 'id': 'tooluse_WtxPHJN1SOOCnMv2_CNe0w', 'index': 1, 'input': '{"tool_input": ""}'}]

titanic
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`
responded: [{'type': 'text', 'text': '\n\nIt looks like we have a single table named "titanic". This table likely contains information about passengers on the Titanic, including survivors. Let\'s check the schema of this table:', 'index': 0}, {'type': 'tool_use', 'name': 'sql_db_schema', 'id': 'tooluse_uyytcjF_RwWosYG9zgjboA', 'index': 1, 'input': '{"table_names": "titanic"}'}]


CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLO

{'input': "what's the average age of survivors",
 'output': [{'type': 'text',
   'text': '\n\nBased on the query results, the average age of survivors on the Titanic was approximately 28.41 years old.\n\nTo provide a more precise answer to the original question: The average age of survivors on the Titanic was 28.41 years old.',
   'index': 0}]}

In [12]:
agent_executor.invoke({"input": "what's the average age of people not survived"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded: [{'type': 'text', 'text': "  Finally, I should construct a query to answer the question.\n\nLet's begin by listing the tables in the database:", 'index': 0}, {'type': 'tool_use', 'name': 'sql_db_list_tables', 'id': 'tooluse_rKiP8DEERau1oe0pJ5WQxw', 'index': 1, 'input': '{"tool_input": ""}'}]

titanic
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`
responded: [{'type': 'text', 'text': '\n\nGreat, we have a table named "titanic". This table likely contains information about passengers on the Titanic, including their survival status and age. Let\'s check its schema:', 'index': 0}, {'type': 'tool_use', 'name': 'sql_db_schema', 'id': 'tooluse_6oB4PhiUTOqvtbREMBKXgQ', 'index': 1, 'input': '{"table_names": "titanic"}'}]


CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Pare

{'input': "what's the average age of people not survived",
 'output': [{'type': 'text',
   'text': '\n\n\n\nNow I can answer the question based on the query results.\n\nThe average age of people who did not survive on the Titanic is approximately 30.14 years old.',
   'index': 0}]}

In [13]:
agent_executor.invoke({"input": "what's is your name "})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded: [{'type': 'text', 'text': "  Finally, I should query the database to answer the question.\n\nI don't know the names of the tables in the database, so I'll start by listing them:", 'index': 0}, {'type': 'tool_use', 'name': 'sql_db_list_tables', 'id': 'tooluse_v3w-oGnNRzKjvj20Ao-GKQ', 'index': 1, 'input': '{"tool_input": ""}'}]

titanic[{'type': 'text', 'text': '\n\nThe database contains only one table: "titanic". This table is unlikely to contain information about my name. As an AI assistant, I don\'t have a personal name, and the information about me wouldn\'t be stored in a database about the Titanic.\n\nGiven this information, I can confidently say:\n\nI don\'t know. The database available to me contains information about the Titanic, which is not related to my name or identity as an AI assistant.', 'index': 0}]

> Finished chain.


{'input': "what's is your name ",
 'output': [{'type': 'text',
   'text': '\n\nThe database contains only one table: "titanic". This table is unlikely to contain information about my name. As an AI assistant, I don\'t have a personal name, and the information about me wouldn\'t be stored in a database about the Titanic.\n\nGiven this information, I can confidently say:\n\nI don\'t know. The database available to me contains information about the Titanic, which is not related to my name or identity as an AI assistant.',
   'index': 0}]}